# Projeto Final de Spark - Nível Básico

## Enviar os dados para o HDFS (através do namenode pelo terminal)


1.  Pelo terminal do WSL2, dentro da pasta spark, baixe o arquivo de dados .rar dentro da pasta spark: 
`sudo curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar`

Obs.: Caso o link veia a ficar inacessível, o arquivo está disponibilizado no diretório [data](https://github.com/cicerooficial/projeto-final-big-data-enginner-sematix/tree/main/data) do GitHub.

2. Instale o unrar para descompactar o arquivo .rar com o comando:`sudo apt install unrar`

3. Descompacte o arquivo .rar com o comando: `sudo unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar`

4. Crie um diretório input dentro da pasta spark: `mkdir input`

5. Envie os arquivos .csv para a pasta input: `sudo mv *.csv /home/cicero/projeto-final-spark/spark/input`

6. Acesse o namenode utilizando o comando `docker exec -it namenode bash`.

7. Crie a pasta projeto-final-spark no HDFS para salvar os arquivos de dados .CSV: `hdfs dfs -mkdir -p /user/cicero/projeto-final-spark`

8. Envie oo arquivos de dados .CSV para a pasta projeto-final-spark no HDFS:
 `hdfs dfs -put /input/*.csv /user/cicero/projeto-final-spark`

9. Confirme se os arquivos foram enviados: `hdfs dfs -ls /user/cicero/projeto-final-spark`



------

## Otimizar todos os dados do hdfs para uma tabela Hive particionada por município

In [194]:
# Criar Sessão Spark
'''
As vantagens de se utilizar a Classe SparkSession é que a mesma fornece acesso às funcionalidades do spark:
•sql Executa as consultas Spark SQL
•catalog Gerenciar tabelas
•read função para ler dados de um arquivo ou outra fonte de dados
•conf objeto para gerenciar configurações de Spark
•sparkContext Core Spark API
'''
from pyspark.sql.functions import *

spark = SparkSession\
.builder\
.appName('Projeto final de Spark - Campanha Nacional de Vacinação contra Covid-19')\
.config('spark.some.config.option', 'some-value')\
.enableHiveSupport()\
.getOrCreate()

In [195]:
#Utilizando o option("inferSchema","true") para que automaticamente o spark identifique os tipos de dados das colunas
#df_painel_covidbr = spark.read.option("sep",";").option("header","true").option("inferSchema","true").csv("hdfs:///user/cicero/projeto-final-spark/")

df_painel_covidbr = spark.read.csv('hdfs://namenode/user/cicero/projeto-final-spark/', 
                                   sep=";", #sep(padrão ,): define o único caractere como separador para cada campo e valor.
                                   header=True, #header(padrão false): usa a primeira linha como nomes de colunas.
                                   inferSchema=True, #inferSchema(padrão false): infere o esquema de entrada automaticamente a partir dos dados. 
                                   ignoreLeadingWhiteSpace=True, #ignoreLeadingWhiteSpace(padrão false): define se os espaços em branco iniciais dos valores que estão sendo lidos devem ser ignorados.
                                   ignoreTrailingWhiteSpace=True) #ignoreTrailingWhiteSpace(padrão false): define se os espaços em branco à direita dos valores que estão sendo lidos devem ser ignorados.

In [196]:
#Visualizar o schema
#df_painel_covidbr.dtypes
df_painel_covidbr.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: decimal(10,0) (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



In [197]:
'''df_painel_covidbr_limpo = df_painel_covidbr.select(col('regiao').cast('String'),
                                                   col('estado').cast('String'),
                                                   col('municipio').cast('String'),
                                                   col('coduf').cast('integer'),
                                                   col('codmun').cast('integer'),
                                                   col('codRegiaoSaude').cast('integer'),
                                                   col('nomeRegiaoSaude').cast('String'),
                                                   col('data').cast('date'),
                                                   col('semanaEpi').cast('integer'),
                                                   col('populacaoTCU2019').cast('String'),
                                                   col('casosAcumulado').cast('integer'),
                                                   col('casosNovos').cast('integer'),
                                                   col('obitosAcumulado').cast('integer'),
                                                   col('obitosNovos').cast('integer'),
                                                   col('Recuperadosnovos').cast('integer'),
                                                   col('emAcompanhamentoNovos').cast('integer'),
                                                   col('interior/metropolitana').cast('integer'))
'''
# Ajustando os dados e removendo a informação de hora, pois todas estão zeradas
df_painel_covidbr_limpo = df_painel_covidbr\
.withColumn('data', functions.from_unixtime(functions.unix_timestamp(df_painel_covidbr.data), 'yyyy-MM-dd'))

#Mostrando o novo dataframe com o campo data ajustado
df_painel_covidbr_limpo.show(3, vertical=True)

-RECORD 0----------------------------
 regiao                 | Brasil     
 estado                 | null       
 municipio              | null       
 coduf                  | 76         
 codmun                 | null       
 codRegiaoSaude         | null       
 nomeRegiaoSaude        | null       
 data                   | 2020-02-25 
 semanaEpi              | 9          
 populacaoTCU2019       | 210147125  
 casosAcumulado         | 0          
 casosNovos             | 0          
 obitosAcumulado        | 0          
 obitosNovos            | 0          
 Recuperadosnovos       | null       
 emAcompanhamentoNovos  | null       
 interior/metropolitana | null       
-RECORD 1----------------------------
 regiao                 | Brasil     
 estado                 | null       
 municipio              | null       
 coduf                  | 76         
 codmun                 | null       
 codRegiaoSaude         | null       
 nomeRegiaoSaude        | null       
 data       

In [198]:
#Otimizar dados por tabelas particionadas por município
spark.sql("CREATE DATABASE IF NOT EXISTS covidbr")
df_painel_covidbr_limpo.write\
.mode('overwrite')\
.partitionBy('municipio')\
.format('csv')\
.saveAsTable('covidbr.municipio', path='hdfs://namenode:8020/user/hive/warehouse/covidbr/')

In [199]:
#Confirmar se os dados foram salvos no diretório
!hdfs dfs -ls 'hdfs://namenode:8020/user/hive/warehouse/covidbr/'

Found 5299 items
-rw-r--r--   2 root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/_SUCCESS
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abadia de Goiás
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abadia dos Dourados
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abadiânia
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abaetetuba
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abaeté
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Abaiara
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/h

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Aragominas
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguacema
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguaiana
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguainha
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguanã
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguapaz
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Araguari
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/co

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Braganey
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Bragança
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Bragança Paulista
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Branquinha
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Brasil Novo
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Brasileira
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Brasilândia
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/wa

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo da Cachoeira
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo da Mata
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo de Minas
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo do Cajuru
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo do Paranaíba
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo do Rio Claro
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Carmo do Rio Verde
drwxr-xr-x   - root supergroup          0 2021-12-10 

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Cristinápolis
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Cristália
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Cristópolis
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Crisólita
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Crisópolis
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Crixás
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Crixás do Tocantins
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapava
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapé
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapé Grande
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapé do Meio
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapé-Açu
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarapé-Miri
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Igarassu
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/w

drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Lavrinhas
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Lavínia
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Leandro Ferreira
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Lebon Régis
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Leme
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Leme do Prado
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Lençóis
drwxr-xr-x   - root supergroup          0 2021-12-10 20:40 hdfs://namenode:8020/user/hive/warehouse/

drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orindiúva
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Oriximiná
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orizona
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orizânia
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orleans
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orlândia
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Orobó
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/munici

drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão Pires
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão Preto
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão Vermelho
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão das Neves
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão do Largo
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão do Pinhal
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=Ribeirão do Sul
drwxr-xr-x   - root supergroup          0 2021-12-10 2

drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Raimundo das Mangabeiras
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Raimundo do Doca Bezerra
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Roberto
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Romão
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Roque
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Roque de Minas
drwxr-xr-x   - root supergroup          0 2021-12-10 20:41 hdfs://namenode:8020/user/hive/warehouse/covidbr/municipio=São Roque do Canaã
drwxr-xr-x   - root supergroup          0 2021

------

## Criar as 3 visualizações pelo Spark com os dados enviados para o HDFS 

In [174]:
# Acessar os dados da tabela municipio do Banco de dados covidbr pelo SparkSQL
spark.sql("SHOW DATABASES").show() #Mostrar os anco de dados

+------------+
|databaseName|
+------------+
|     covidbr|
|     default|
+------------+



In [175]:
# Selecionar o banco de dados covidbr 
spark.sql("USE covidbr")

# Acessar os dados da tabela municipio do Banco de dados covidbr pelo SparkSQL
spark.sql("SHOW TABLES").show() #Mostrar os anco de dados

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| covidbr|municipio|      false|
+--------+---------+-----------+



### Visualização 1 - Casos Recuperados | Em Acompanhamento
Obs.: Os resultados serão extraídos a partir da última data registrada que foi em 2021-07-06.

In [221]:
## 1.1 Casos Recuperados
casos_recuperados = spark.sql("SELECT Recuperadosnovos as Casos_Recuperados FROM municipio order by 1 desc limit 1")
casos_recuperados.show()

## 1.2 Em Acompanhamento
em_acompanhamento = spark.sql("SELECT emAcompanhamentoNovos as Em_Acompanhamento FROM municipio WHERE order by 1 desc limit 1")
em_acompanhamento.show()


+-----------------+
|Casos_Recuperados|
+-----------------+
|         17262646|
+-----------------+

+-----------------+
|Em_Acompanhamento|
+-----------------+
|          1317658|
+-----------------+



### Visualização 2 - Casos Acumulado | Casos Novos | Incidência
Obs.: **Incidência:** Quantidade proporcional de casos confirmados para cada 1 milhão de habitantes. A população considerada no cálculo é a estimativa populacional para 2019 do Instituto Brasileiro de Geografia e Estatística (IBGE), salvo quando especificada referência distinta. **Método de cálculo: (casos confirmados * 1.000.000) / população**.
Fonte: https://www.coronavirus.sc.gov.br/sobre-os-dados/

In [230]:
#2.1 Acumulado
casos_acumulados = spark.sql("SELECT casosAcumulado as Casos_Acumulados FROM municipio order by 1 desc limit 1")
casos_acumulados.show()

#2.2 Casos novos
casos_novos = spark.sql("SELECT casosNovos as Casos_Novos FROM municipio order by 1 desc limit 1")
casos_novos.show()

#2.3 Incidência (Casos acumulados)
incidencia = spark.sql("SELECT  cast(((casosAcumulado*1000000)/populacaoTCU2019) as decimal(5,1)) as Incidencia FROM municipio order by 1 desc limit 1")
incidencia.show()


+----------------+
|Casos_Acumulados|
+----------------+
|        18855015|
+----------------+

+-----------+
|Casos_Novos|
+-----------+
|     115228|
+-----------+

+----------+
|Incidencia|
+----------+
|    9999.8|
+----------+



### Visualização 3 - Óbtos Acumulados | Óbtos Novos | Letalidade | Mortalidade
Obs.1: **Letalidade:** Taxa percentual de casos confirmados que evoluíram a óbito. **Método de cálculo: (número de óbitos x 100) / número de casos confirmados**.

Obs.2: **Mortalidade:** Quantidade proporcional de óbitos para cada 1 milhão habitantes, seguindo a mesma metodologia de cálculo da incidência de casos confirmados. **Método de cálculo: (óbitos * 1.000.000) / população**.
Fonte: https://www.coronavirus.sc.gov.br/sobre-os-dados/

In [231]:
#3.1 Óbitos acumulados
obitos_acumulados = spark.sql("SELECT obitosAcumulado as Obitos_Acumulados FROM municipio order by 1 desc limit 1")
obitos_acumulados.show()

#3.2 Obitos novos
obitos_novos = spark.sql("SELECT obitosNovos as Obitos_Novos FROM municipio order by 1 desc limit 1")
obitos_novos.show()

#3.3 Letalidade
letalidade = spark.sql("SELECT cast(((obitosAcumulado * 100) / casosAcumulado) as decimal(5,1)) as Letalidade FROM municipio order by 1 desc limit 1")
letalidade.show()

#3.4 Mortalidade
mortalidade = spark.sql("SELECT cast(((obitosAcumulado * 1000000) / populacaoTCU2019) as decimal(4,1)) as Mortalidade FROM municipio order by 1 desc limit 1")
mortalidade.show()

+-----------------+
|Obitos_Acumulados|
+-----------------+
|           526892|
+-----------------+

+------------+
|Obitos_Novos|
+------------+
|        4249|
+------------+

+----------+
|Letalidade|
+----------+
|    1500.0|
+----------+

+-----------+
|Mortalidade|
+-----------+
|      999.9|
+-----------+



------

## Salvar a primeira visualização como tabela Hive

## Salvar a segunda visualização com formato parquet e compressão snappy

## Salvar a terceira visualização em um tópico no Kafka

## Criar a visualização pelo Spark com os dados enviados para o HDFS

------

### Fontes de consultas:

##### pyspark.sql module:  https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#module-pyspark.sql.functions

##### Spark SQL, DataFrames and Datasets Guide: https://spark.apache.org/docs/2.0.2/sql-programming-guide.html#sql

##### Spark SQL Guide: https://spark.apache.org/docs/latest/sql-getting-started.html

##### Como obter no SQL o último registro inserido de acordo com sua data: https://pt.stackoverflow.com/questions/13125/como-obter-no-sql-o-%C3%BAltimo-registro-inserido-de-acordo-com-sua-data

##### Sobre os dados do Coronavirus: https://www.coronavirus.sc.gov.br/sobre-os-dados/

##### Projeção da população do Brasil: https://www.ibge.gov.br/apps/populacao/projecao/index.html

##### CAST para DECIMAL no MySQL: https://stackoverflow.com/questions/11830509/cast-to-decimal-in-mysql